# Optimization of OT2 transfer paramters of viscous liquids guided by IPOPT optimization

## Summary

This notebook objective is to generate new suggestions of aspiration and dispense rates that will minimize the tansfer error while minimizing the time of transfer of a viscous liquid. The code in this notebook will aim to optimize the liquid transfer parameters for a specific volume to minimize the error in transfer and the time of transfer to dispense 1000 uL.
The code is strucutred as follows:
1.  Fisrt section is for importing the relevant packages to perform IPOPT, inclduing Scikit-learn and Pyomo
2.  Second section includes the definition of the IPOPT_LiqTransfer class that includes the method optimized_suggestions() that generates IPOPT optimized aspiration and dispense rate values for a particular data set.
3. Third section includes the code to run in conjuction with the OT2 viscous liquid transfer notebook. The steps for the optimziation are:
    i. Initilize a IPOPT_LiqTransfer objecet and load initilization data using data_from_csv() method

    ii. Run optimized_suggestions() method

    iii. Run liquid transfer gravimetric experiment using the best suggestion for aspiration and dispense rates

    iv. Update latest %error obtained from the transfer using suggested aspiration and dispense rates.
    
    v. Iterate through steps ii-iV

## 1. Imports

In [1]:
#%% General Imports
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt

#sklearn imports
import sklearn.linear_model as linear_model

#pyomo
from pyomo.environ import *

In [2]:
#Find directory of repository to generalize paths
REPO = 'viscosity_liquid_transfer_Pablo'
parent_path = os.getcwd().split(REPO)[0]

## 2. IPOPT_LiqTransfer class

In [3]:
class IPOPT_LiqTransfer:

    def __init__(self, liquid_name):
        self.liquid_name = liquid_name
        self._data = None
        self.features = ['aspiration_rate','dispense_rate']
        self.objectives = ['%error']
        self.bmax = 1.25
        self.bmin = 0.1
        self._latest_suggestion = None
        self._latest_volume = None
    
    def set_data(self,df):
        df['time_asp_1000'] = 1000/df['aspiration_rate'] + 1000/df['dispense_rate'] + df['delay_aspirate'] + df['delay_dispense']
        self._data = df

    
    def data_from_csv(self,file_name):
        data = pd.read_csv(file_name)
        data = data.loc[:,['liquid','pipette','volume','aspiration_rate','dispense_rate','blow_out_rate','delay_aspirate','delay_dispense','delay_blow_out','%error']]
        self.set_data(data)

    def update_data(self,error):
        updated_data = pd.concat([self._data,self._data.iloc[[-1]]],ignore_index=True)
        updated_data.loc[updated_data.last_valid_index(),'volume'] = self._latest_volume
        updated_data.loc[updated_data.last_valid_index(),'aspiration_rate']  = self._latest_suggestion['aspiration_rate'][0]
        updated_data.loc[updated_data.last_valid_index(),'dispense_rate']  = self._latest_suggestion['dispense_rate'][0]
        updated_data.loc[updated_data.last_valid_index(),'%error'] = error
        self.set_data(updated_data)
        return self._data
                                
    def xy_split(self,volume=1000):
        df_train = self._data.where(self._data['volume']==volume).dropna(how='all').copy()
        x_train = df_train[self.features]
        y_train = df_train[self.objectives]
        return x_train,y_train

    def set_bounds(self, x_train):
        return x_train.iloc[0,0]*self.bmin, x_train.iloc[0,0]*self.bmax

    def fit_lin(self,volume=1000):
        lin_model = linear_model.LinearRegression()
        x_train,y_train = self.xy_split(volume=volume)
        
        min,max = self.set_bounds(x_train)

        lin_model.fit(x_train,y_train)


        m1,m2 = lin_model.coef_.tolist()[0]
        b= lin_model.intercept_.tolist()[0]

        return m1,m2,b,min,max
    
    
    def optimized_suggestions(self,volume= 1000):
        self._latest_volume = volume        
        
        model = ConcreteModel()
        m1,m2,b,min,max = self.fit_lin(volume=volume)

        def obj_time_for_1000(m):
    
            return 1000/m.x1 + 1000/m.x2 + 10
        # # Define decision variables

        model.x1 = Var(initialize= (min+max)/2, bounds = (min,max))
        model.x2 = Var(initialize= (min+max)/2, bounds = (min,max))

        model.obj = Objective(rule= obj_time_for_1000, sense=minimize)
        
        # Define constraints
        model.constraints = ConstraintList()
        model.constraints.add(expr= m1 * model.x1 + m2 * model.x2 + b <= 2.5)
        model.constraints.add(expr= m1 * model.x1 + m2 * model.x2 + b >= -2.5)
        model.constraints.add(expr= model.obj >= 10)

        solver = SolverFactory('ipopt')
        solver.solve(model)
        
        self._latest_suggestion = pd.DataFrame({'aspiration_rate':model.x1.value,'dispense_rate':model.x2.value}, index=[0])
        
        return self._latest_suggestion
        

        

## 3. Guided IPOPT optimization
i.   Create IPOPT_LiqTransfer object and load data set.

Please set liquid name and volume to transfer according to the experiment.

In [20]:
# Change according to experiment
liquid_name = 'Viscosity_std_204' 
experiment = '1_vol_opt_duplicate_unused' #or '3_vol_opt' 

# Do not change
liq = IPOPT_LiqTransfer(liquid_name)
liq.data_from_csv(parent_path+REPO+'\\Opentrons_experiments\\BOTorch_optimization\\CCF_initialization\\BOTorch_optimization_CCF_' +liquid_name+'.csv')
liq._data

,liquid,pipette,volume,aspiration_rate,dispense_rate,blow_out_rate,delay_aspirate,delay_dispense,delay_blow_out,%error,time_asp_1000
0,Viscosity_std_204,p1000,1000.0,89.911952,89.911952,0,5,5,0,-10.047459,32.243984
1,Viscosity_std_204,p1000,500.0,89.911952,89.911952,0,5,5,0,-5.313115,32.243984
2,Viscosity_std_204,p1000,300.0,89.911952,89.911952,0,5,5,0,-5.698962,32.243984
3,Viscosity_std_204,p1000,1000.0,112.389940,8.991195,0,5,5,0,2.812826,130.117512
4,Viscosity_std_204,p1000,500.0,112.389940,8.991195,0,5,5,0,4.734344,130.117512
5,Viscosity_std_204,p1000,300.0,112.389940,8.991195,0,5,5,0,3.599954,130.117512
6,Viscosity_std_204,p1000,1000.0,112.389940,112.389940,0,5,5,0,-12.026855,27.795187
7,Viscosity_std_204,p1000,500.0,112.389940,112.389940,0,5,5,0,-7.211483,27.795187
8,Viscosity_std_204,p1000,300.0,112.389940,112.389940,0,5,5,0,-4.232743,27.795187
9,Viscosity_std_204,p1000,1000.0,8.991195,112.389940,0,5,5,0,-11.193425,130.117512


ii.   Run IPOPT_LiqTransfer.optimized_suggestions() method to obtain optimized aspiration and dispense rates.

In [28]:
volume = 1000
liq.optimized_suggestions(volume=volume)

,aspiration_rate,dispense_rate
0,112.38994,43.204816


iii. Run liquid transfer using the best suggestion for aspiration and dispense rates in OT2 notebook.

iv. Update latest %error obtained from the transfer using suggested aspiration and dispense rates.

In [27]:
liq.update_data(-5.928784)

,liquid,pipette,volume,aspiration_rate,dispense_rate,blow_out_rate,delay_aspirate,delay_dispense,delay_blow_out,%error,time_asp_1000
0,Viscosity_std_204,p1000,1000.0,89.911952,89.911952,0,5,5,0,-10.047459,32.243984
1,Viscosity_std_204,p1000,500.0,89.911952,89.911952,0,5,5,0,-5.313115,32.243984
2,Viscosity_std_204,p1000,300.0,89.911952,89.911952,0,5,5,0,-5.698962,32.243984
3,Viscosity_std_204,p1000,1000.0,112.389940,8.991195,0,5,5,0,2.812826,130.117512
4,Viscosity_std_204,p1000,500.0,112.389940,8.991195,0,5,5,0,4.734344,130.117512
5,Viscosity_std_204,p1000,300.0,112.389940,8.991195,0,5,5,0,3.599954,130.117512
6,Viscosity_std_204,p1000,1000.0,112.389940,112.389940,0,5,5,0,-12.026855,27.795187
7,Viscosity_std_204,p1000,500.0,112.389940,112.389940,0,5,5,0,-7.211483,27.795187
8,Viscosity_std_204,p1000,300.0,112.389940,112.389940,0,5,5,0,-4.232743,27.795187
9,Viscosity_std_204,p1000,1000.0,8.991195,112.389940,0,5,5,0,-11.193425,130.117512


v. Iterate through last two code cells.

In [ ]:
#save after each standard-experiment iteration
liq._data.to_csv(parent_path+REPO+'\\Opentrons_experiments\\BOTorch_optimization\\VS_code_csv\\IPOPT_'+liquid_name+'_'+experiment+'.csv', index = False)